## 2 Ground Control Stations (simple)

This is a first version. The two ground control stations are working in series. 

In [14]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State
from env import Enviroment,GCS

kill_processes()

## Create Plans

In [15]:
offsets = [  # east, north, up, heading
    (5, 5, 0, 90),
    (10, -2, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]
n_vehicles = len(offsets)
side_lens = (5, 7, 4, 3, 2)  # must agree with n_vehicles
local_paths = [
    Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens
]
plans = [Plan.basic(wps=path, alt=5, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [16]:
homes = np.array([offset[:3] for offset in offsets])
global_paths = [
    local2global(np.array(path), home, pairwise=True)
    for path, home in zip(local_paths, homes)
]
colors = ["Green", "Blue", "Red", "Orange", "Yellow"]  # must agree with n_vehicles
waypoints = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [17]:
simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path="simulators/gazebo/worlds/runway.world",
    vehicle_models=n_vehicles * ["drone"],
    markers=waypoints,
)

Launch Simulator

In [18]:
uavs = simulator.launch()
env = Enviroment(uavs)
gcs1 = GCS(uavs[:2])
gcs2 = GCS(uavs[2:])

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀
Vehicle 5 launched 🚀


## Execute Plan

In [19]:
while env.vehs:
    env.gather_broadcasts()
    gcs1.gather_broadcasts()
    gcs1.save_pos()
    gcs2.gather_broadcasts()
    gcs2.save_pos()
    for uav in env.vehs.copy():
        env.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            env.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 4: ▶️ Starting Step: Check disarmed
Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 3: ▶️ Starting Step: Check disarmed
Vehicle 5: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 3: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 3: ▶️ Starting Step: Check EKF
Vehicle 4: ✅ Step: Check disarmed is done
Vehicle 4: ▶️ Starting Step: Check EKF
Vehicle 5: ✅ Step: Check disarmed is done
Vehicle 5: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 5: ✅ Step: Check EKF is done
Vehicle 5: ▶️ Starting Step: Check GPS
Vehicle 4: ✅ Step: Check EKF is done
Vehicle 4: ▶️ Starting Step: Check GPS
Vehicle 3: ✅ Step: Check EKF is done
Vehicle 3: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check

In [20]:
paths_gs1 = {
    f"uav_{i+1}": {"pos": np.stack(path), "color": colors[i]}
    for i,path in enumerate(gcs1.paths.values())
}
plot_3d_interactive(
    paths_gs1, title="gcs1", expand=[0.2, 0.2, 0.6], ground=-0.05
)

In [21]:
paths_gs2 = {
    f"uav_{i+3}": {"pos": np.stack(path), "color": colors[2+i]}
    for i,path in enumerate(gcs2.paths.values())
}
plot_3d_interactive(
    paths_gs2, title="gcs2", expand=[0.2, 0.2, 0.6], ground=-0.05
)

In [22]:
uavs[2].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to  -> (0, 4, 5)'>
    ✅ <Step 'go to  -> (4, 4, 5)'>
    ✅ <Step 'go to  -> (4, 0, 5)'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
  ✅ <Action 'LAND'>
    ✅ <Step 'land'>